In [20]:
import mediapipe as mp
import numpy as np
import cv2

import ctypes

WINDOW_NAME = 'Full Integration'


# initialize video capture object to read video from external webcam
video_capture = cv2.VideoCapture(1)
# if there is no external camera then take the built-in camera
if not video_capture.read()[0]:
    video_capture = cv2.VideoCapture(0)
    

# Full screen mode
# cv2.namedWindow(WINDOW_NAME, cv2.WND_PROP_FULLSCREEN)
# cv2.setWindowProperty(WINDOW_NAME, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

# cap = cv2.VideoCapture(0)

hands = mp.solutions.hands
facmesh = mp.solutions.face_mesh
face = facmesh.FaceMesh()
hands_mesh = hands.Hands()
draw = mp.solutions.drawing_utils


h = 0
w = 0
while True:

# _, frm = cap.read()
    # get Screen Size
    user32 = ctypes.windll.user32
    screen_width, screen_height = user32.GetSystemMetrics(0), user32.GetSystemMetrics(1)
 
 # read video frame by frame
    ret, frame = video_capture.read()

    frame = cv2.flip(frame, 1)

    frame_height, frame_width, _ = frame.shape

    scaleWidth = float(screen_width)/float(frame_width)
    scaleHeight = float(screen_height)/float(frame_height)

    if scaleHeight>scaleWidth:
        imgScale = scaleWidth

    else:
        imgScale = scaleHeight

    newX,newY = frame.shape[1]*imgScale, frame.shape[0]*imgScale

    frm = cv2.resize(frame,(int(newX),int(newY)))
    rgb = cv2.cvtColor(frm, cv2.COLOR_BGR2RGB)
    # frm.shape[1], frm.shape[0]
      
    op1 = face.process(rgb)
    op2 = hands_mesh.process(rgb)

    if op2.multi_hand_landmarks and op1.multi_face_landmarks:
        for handLms, faceLms in zip(op2.multi_hand_landmarks, op1.multi_face_landmarks):
            show_fmesh = False
            p0x = 0
            p4x = 0
            for id, lm in enumerate(handLms.landmark):
                if id == 0:
                    p0x = lm.x
                if id == 4:
                    p4x = lm.x
                if p4x > p0x:
                    show_fmesh = True
                    cv2.rectangle(frm, (0,0), (int(newX), int(newY)), (0,0,0), -1)
            draw.draw_landmarks(frm, handLms, hands.HAND_CONNECTIONS)
            if show_fmesh:
                draw.draw_landmarks(frm, faceLms, facmesh.FACE_CONNECTIONS, landmark_drawing_spec=draw.DrawingSpec(color=(0,128,0), circle_radius=1)) 
            

    cv2.imshow("window", frm)

    if cv2.waitKey(1) == 27:
        video_capture.release()
        cv2.destroyAllWindows()
        break